In [71]:
from Bio import AlignIO
import numpy as np
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

In [7]:
ip = 'HIV-p6-gapped.fasta'

In [8]:
records = AlignIO.read(ip, "fasta")

In [9]:
records

<<class 'Bio.Align.MultipleSeqAlignment'> instance (5218 records of length 763) at 10ceb81a0>

In [10]:
records.get_alignment_length()

763

In [11]:
len(records)

5218

In [12]:
print(records)

Alignment with 5218 rows and 763 columns
------------------------------------------MG...--- B.FR.83.HXB2_LAI_IIIB_BRU_K03455
--------------------------------------------...--- B.FR.83.HXB2_LAI_IIIB_BRU.K03455
--------------------------------------------...--- A.CD.87.P4039.MH705157
--------------------------------------------...--- A.CH.03.HIV_CH_BID_V3538_2003.JQ403028
--------------------------------------------...--- A.ZA.04.04ZASK162B1.DQ396400
--------------------------------------------...--- A1.AU.03.PS1044_Day0.DQ676872
--------------------------------------------...--- A1.BG.09.V_09_008.MH746258
--------------------------------------------...--- A1.BG.09.V_09_017.MH746253
--------------------------------------------...--- A1.CD.02.LA01AlPr.KU168256
--------------------------------------------...--- A1.CD.87.2106.MH705158
--------------------------------------------...--- A1.CD.87.70641.MH705151
--------------------------------------------...--- A1.CD.87.PBS6126.MH705153
------

In [34]:
ids = [rec.id for rec in records]

In [22]:
arr = np.array([list(str(rec.seq)) for rec in records])

In [31]:
keep=[]
for c in range(arr.shape[1]):
    counter=0
    for r in range(arr.shape[0]):
        if arr[r,c]=='-':
            counter+=1
    if counter<0.67*arr.shape[0]:
        keep.append(c)

In [45]:
new_records=[]
for rec in records:
    new_seq = ''.join(rec[i] for i in keep)
    new_item= SeqRecord(Seq(new_seq), id=rec.id, description=rec.description)
    new_records.append(new_item)

In [48]:
filtered_alignment = MultipleSeqAlignment(new_records)
AlignIO.write(filtered_alignment, 'op.fasta', "fasta")

1

In [50]:
from Bio.Align import AlignInfo

In [72]:
msa = AlignIO.read('op.fasta', "fasta")

In [55]:
summary_align = AlignInfo.SummaryInfo(msa)

In [62]:
consensus = summary_align.dumb_consensus(threshold=0.0, ambiguous='-', require_multiple=1)
consensus

Seq('LQSRPEPTAPPAESFRFGEETTTPSQKQEQKDKELYPLTSLKSLFGNDPLSQ*')

In [70]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from collections import Counter

In [69]:
con_chars = []
for i in range(msa.get_alignment_length()):
    col = [rec.seq[i] for rec in msa]
    counts = Counter(col)
    con_res = counts.most_common(1)[0][0]
    con_chars.append(con_res)

consensus_sequence = ''.join(con_chars)
print(consensus_sequence)

LQSRPEPTAPPAESFRFGEET-TPSQKQEQKDKELYPLTSLKSLFGNDPLSQ*


In [79]:
for rec in msa:
    print(rec.seq)
    ct = rec.seq.find(ss)
    break

LQSRPEPTAPPEESFRSGVETTTPPQKQEPIDKELYPLTSLRSLFGNDPSSQ*


In [81]:
ct != -1

True

In [85]:
ss = 'KELYPLASL'

In [82]:
len(msa)

5218

In [86]:
count = 0
for rec in msa:
    if rec.seq.find(ss) != -1:
        count+=1
print(count/len(msa))

0.12916826370256804


In [87]:
ss = 'LQSRPEPTAPPAESFSFGEETATPAQKQEPIDQEKYPLTALKSLFGNDPLSQ*'

In [88]:
ssl = list(ss)
sims = []
for rec in msa:
    y = [s==r for s,r in zip(ssl, list(str(rec.seq)))]
    sims.append(sum(y))

In [90]:
len(sims)

5218

In [124]:
high = np.argsort(sims)[::-1]

In [125]:
high

array([ 363, 1735, 3627, ..., 5204, 5209, 5190], shape=(5218,))

In [127]:
h

[363, 1735, 3627]

In [126]:
h = [int(ind) for ind in high[:3]]

In [128]:
for i in h:
    print(msa[i].seq)

LQSRPEPTAPPAESFSFGGETTTPAQKQEPIDQEKYPLTALKSLFGNDPLSQ*
LQSRPEPTAPPAESFRFGEETATPAQKQEPIDKELYPLTSLKSLFGNDPSSQ*
LQSRPEPTAPPAESFRFGEETTTPSQKQEPIDKELYPLTSLKSLFGNDPLSQ*


In [132]:
%%bash
python3 epitope-fraction.py op.fasta KELYPLASL

0.12916826370256804


In [167]:
ss = 'PQSRPEPSAPPAENFGMGEEI-TPSLKQEQKNREQPPSISLKSLFGNDPLSQ*'
ssl = list(ss)

In [195]:
msa = AlignIO.read('op.fasta', "fasta")

groupedseqs = defaultdict(list)               # dictionary w lists as vals
for rec in msa:                               # for each record in msa
    subtype = (rec.id).split('.')[0]          # split each record id by . and take first item (subtype str)
    groupedseqs[subtype].append(rec)          # append that record to list under dictionary key for the subtype
    
mean_diffs = {}                              # empty dict

for subtype, records in groupedseqs.items():  # iter. thru subtypes, records in dict of subtype:records, where records is a list
    matches=[]                                # init. matches list
    for val in records:                       # for each record in the list
        m = [s==r for s,r in zip(ssl, list(str(val.seq)))]   # dist = list of booleans saying if each char in rec.seq is diff. from char in ss
        matches.append(sum(m))                # add dist for each seq under that subtype to distance list.
    mean_diffs[subtype] = np.mean(matches)    # calculate mean of the distances list: total dist across seqs/# of seqs
sortdiffs = sorted(mean_diffs.items(), key=lambda x: x[1], reverse=True) # sort the dict via index [1] of (subtype,mean) tuples, descending. 
                                              # key here is arg for the method of sorting, not dict key
print(sortdiffs[0][0])

03_A6B


In [197]:
sortdiffs

[('03_A6B', np.float64(48.5)),
 ('A6', np.float64(46.91150442477876)),
 ('22A1U', np.float64(45.0)),
 ('AJ', np.float64(45.0)),
 ('100_01C', np.float64(44.666666666666664)),
 ('115_01C', np.float64(44.0)),
 ('A1DHK', np.float64(44.0)),
 ('A3', np.float64(43.333333333333336)),
 ('52_01B', np.float64(43.333333333333336)),
 ('22_01A1', np.float64(43.27272727272727)),
 ('45_cpx', np.float64(43.0)),
 ('59_01B', np.float64(43.0)),
 ('A6B', np.float64(42.666666666666664)),
 ('AU', np.float64(42.666666666666664)),
 ('68_01B', np.float64(42.0)),
 ('102_0107', np.float64(42.0)),
 ('149_01B', np.float64(42.0)),
 ('01ADF2', np.float64(42.0)),
 ('103_01B', np.float64(41.857142857142854)),
 ('79_0107', np.float64(41.666666666666664)),
 ('01_AE', np.float64(41.39959016393443)),
 ('35_A1D', np.float64(41.26315789473684)),
 ('67_01B', np.float64(41.0)),
 ('80_0107', np.float64(41.0)),
 ('81_cpx', np.float64(41.0)),
 ('114_0155', np.float64(41.0)),
 ('0102', np.float64(41.0)),
 ('01G', np.float64(41.0))

In [184]:
from Bio import AlignIO
from collections import defaultdict
import numpy as np
import sys
#ip = sys.argv[1]
ss = 'PQSRPEPSAPPAENFGMGEEI-TPSLKQEQKNREQPPSISLKSLFGNDPLSQ*'
ssl = list(ss)

msa = AlignIO.read('op.fasta', "fasta")

groupdiffs = defaultdict(list)
for rec in msa:
    group = (rec.id).split('.')[0]
    dist = [s==r for s,r in zip(ssl, list(str(val.seq)))]
    groupdiffs[group].append(sum(dist))
    
avgdiffs = {g: np.mean(vals) for g, vals in groupdiffs.items()}
sortdiffs = sorted(avgdiffs.items(), key=lambda x: x[1], reverse=True)
print(sortdiffs[0])


('B', np.float64(1.0))


In [185]:
sortdiffs

[('B', np.float64(1.0)),
 ('A', np.float64(1.0)),
 ('A1', np.float64(1.0)),
 ('A2', np.float64(1.0)),
 ('A3', np.float64(1.0)),
 ('A4', np.float64(1.0)),
 ('A6', np.float64(1.0)),
 ('A7', np.float64(1.0)),
 ('A8', np.float64(1.0)),
 ('C', np.float64(1.0)),
 ('D', np.float64(1.0)),
 ('F1', np.float64(1.0)),
 ('F2', np.float64(1.0)),
 ('G', np.float64(1.0)),
 ('H', np.float64(1.0)),
 ('J', np.float64(1.0)),
 ('K', np.float64(1.0)),
 ('L', np.float64(1.0)),
 ('01_AE', np.float64(1.0)),
 ('02_AG', np.float64(1.0)),
 ('03_A6B', np.float64(1.0)),
 ('04_cpx', np.float64(1.0)),
 ('05_DF', np.float64(1.0)),
 ('06_cpx', np.float64(1.0)),
 ('07_BC', np.float64(1.0)),
 ('08_BC', np.float64(1.0)),
 ('09_cpx', np.float64(1.0)),
 ('10_CD', np.float64(1.0)),
 ('11_cpx', np.float64(1.0)),
 ('12_BF', np.float64(1.0)),
 ('13_cpx', np.float64(1.0)),
 ('14_BG', np.float64(1.0)),
 ('15_01B', np.float64(1.0)),
 ('16_A2D', np.float64(1.0)),
 ('17_BF1', np.float64(1.0)),
 ('18_cpx', np.float64(1.0)),
 ('19_cpx